In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup, NavigableString
import re
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
from datetime import datetime
import boto3
import pickle

### The objective of this web scrapper is to collect the data of all the real state information available in Luxembourg.

There are mainly 2 sources:
1. Athome (I will focus on this now)
2. Luxhouse whatever Idk now

1. Ideally we want to record all the changes in the houses on the daily basics so we can see if those are being bought or new houses are being added.

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
def clean_title(tag):
    # Concatenate all text in the span, separating them with spaces
    return ' '.join(filter(None, [text.strip() for text in tag.stripped_strings]))

def get_entity_details(link_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, como Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    
    try:
        response = requests.get(link_url, headers=headers)
        url = response.url
        print(url)
        response.raise_for_status()  # Raise an error for bad status codes
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract title
        title_tag = soup.find('span', {'class': 'property-card-immotype-title'})
        if title_tag:
            title = clean_title(title_tag)
            # Remove unwanted text (like "Ref <some number>")
            title = re.sub(r'Ref\s*\d+', '', title).strip()
        else:
            title = 'N/A'

        # Extract description
        description_div = soup.find('div', class_='DescriptionStyle__DescriptionContentDropdown-sc-v1zyng-5')
        if description_div:
            paragraph = description_div.find('p')
            if (paragraph):
                description_text = paragraph.get_text(strip=True)
        else:
            description_text = 'N/A'

        # Extract price
        price_tag = soup.find('span', {'class': 'property-card-price'})
        price = price_tag.get_text(strip=True) if price_tag else 'N/A'

        # Extract details
        rooms_tag = soup.find('li', {'class': 'item-rooms'})
        rooms = rooms_tag.find('span').get_text(strip=True) if rooms_tag else 'N/A'

        bathrooms_tag = soup.find('li', {'class': 'item-bathrooms'})
        bathrooms = bathrooms_tag.find('span').get_text(strip=True) if bathrooms_tag else 'N/A'

        garages_tag = soup.find('li', {'class': 'item-garages'})
        garages = garages_tag.find('span').get_text(strip=True) if garages_tag else 'N/A'

        surface_tag = soup.find('li', {'class': 'item-surface'})
        surface = surface_tag.find('span').get_text(strip=True) if surface_tag else 'N/A'

        # Extract address
        address_tag = soup.find('div', class_='block-localisation-address')
        address = address_tag.get_text(strip=True) if address_tag else 'N/A'
        
        # Agency

        agency_tag = soup.find('div', class_='agency-details__name')
        agency = agency_tag.get_text(strip=True) if agency_tag else 'N/A'


        # Extract image links
        image_tags = soup.select('ul.GalleryTheme__GalleryList-sc-140omi6-4 li div.square a picture img')
        image_links = [img['src'] for img in image_tags]
        image_count = len(image_links)

        # Extract characteristics
        characteristics_blocks = soup.find_all('div', class_='characteristics-block')
        characteristics = {}

        for block in characteristics_blocks:
            block_title = block.find('div', class_='characteristics-title').get_text(strip=True)
            items = block.find_all('div', class_='characteristics-item')

            for item in items:
                label = item.find('span', class_='characteristics-item-label').get_text(strip=True)
                value = item.find('span', class_='characteristics-item-value').get_text(strip=True)
                key = f"{block_title}_{label}"
                characteristics[key] = value

        # Create a dictionary of the details
        details = {
            "link_url": url,
            "title": title,
            "description": description_text,
            "price": price,
            "rooms": rooms,
            "bathrooms": bathrooms,
            "garages": garages,
            "surface": surface,
            "address": address,
            "agency": agency, 
            "image_links": image_links,
            "image_count": image_count
        }

        # Add characteristics to details
        details.update(characteristics)

        return url, details

    except Exception as e:
        return link_url, {"link": link_url, "error": str(e)}

def build_dataframe(links):
    data = []
    with ThreadPoolExecutor(max_workers=100) as executor:
        future_to_link = {executor.submit(get_entity_details, link): link for link in links}
        for future in as_completed(future_to_link):
            link = future_to_link[future]
            try:
                link, result = future.result()
                data.append(result)
            except Exception as e:
                data.append({"link": link, "error": str(e)})
    
    df = pd.DataFrame(data)
    return df


In [4]:
def format_price_and_surface(result_df):
    result_df.rename(columns={'price':'price_raw','surface':'surface_raw','address':'address_raw'},inplace=True)
    
    # Step 1: Remove unwanted characters and create a cleaned price column
    result_df['price_cleaned'] = result_df['price_raw'].astype(str).str.replace(' ', '').str.replace('€', '').str.replace('\u202f', '').str.replace('\xa0', '').str.replace('\x00', '')

    # Step 2: Determine if the price is absolute or an interval
    result_df['is_price_interval'] = result_df['price_cleaned'].str.contains('De')

    # Step 3: Extract absolute prices
    result_df['absolute_price'] = np.where(
        result_df['is_price_interval'],
        np.nan,
        result_df['price_cleaned'].str.extract('(\d+)', expand=False).astype(float)
    )

    # Step 4: Extract interval prices without using regex
    def extract_interval(price):
        if isinstance(price, str) and 'De' in price and 'à' in price:
            start, end = price.split('à')
            start = start.replace('De', '')
            return float(start), float(end)
        return np.nan, np.nan

    intervals = result_df['price_cleaned'].apply(lambda x: extract_interval(x))
    result_df['interval_start'] = intervals.apply(lambda x: x[0])
    result_df['interval_end'] = intervals.apply(lambda x: x[1])

    # Step 5: Calculate interval average
    result_df['interval_average'] = result_df[['interval_start', 'interval_end']].mean(axis=1)

    # Step 6: Replace NaN values with appropriate indicators
    result_df['interval_start'] = result_df['interval_start'].replace({np.nan: None})
    result_df['interval_end'] = result_df['interval_end'].replace({np.nan: None})
    result_df['interval_average'] = result_df['interval_average'].replace({np.nan: None})

    # Process surface columns (new part)
    # Step 1: Remove unwanted characters and create a cleaned surface column
    result_df['surface_cleaned'] = result_df['surface_raw'].astype(str).str.replace(' ', '').str.replace('m²', '').str.replace('\u202f', '').str.replace('\xa0', '')

    # Step 2: Determine if the surface is absolute or an interval
    result_df['is_surface_interval'] = result_df['surface_cleaned'].str.contains('De')

    # Step 3: Extract absolute surfaces
    result_df['absolute_surface'] = np.where(
        result_df['is_surface_interval'],
        np.nan,
        result_df['surface_cleaned'].str.extract('(\d+)', expand=False).astype(float)
    )

    # Step 4: Extract interval surfaces without using regex
    def extract_surface_interval(surface):
        if isinstance(surface, str) and 'De' in surface and 'à' in surface:
            start, end = surface.split('à')
            start = start.replace('De', '')
            return float(start), float(end)
        return np.nan, np.nan

    surface_intervals = result_df['surface_cleaned'].apply(lambda x: extract_surface_interval(x))
    result_df['surface_interval_start'] = surface_intervals.apply(lambda x: x[0])
    result_df['surface_interval_end'] = surface_intervals.apply(lambda x: x[1])

    # Step 5: Calculate interval average
    result_df['surface_interval_average'] = result_df[['surface_interval_start', 'surface_interval_end']].mean(axis=1)

    # Step 6: Replace NaN values with appropriate indicators
    result_df['surface_interval_start'] = result_df['surface_interval_start'].replace({np.nan: None})
    result_df['surface_interval_end'] = result_df['surface_interval_end'].replace({np.nan: None})
    result_df['surface_interval_average'] = result_df['surface_interval_average'].replace({np.nan: None})
    
    return result_df


In [7]:
def main():
    with open('list_of_links.pkl', 'rb') as f:
        all_links = pickle.load(f)
        
    df_start = pd.DataFrame(all_links, columns=["base"])
    df_start["all_links"] = "https://www.athome.lu" + df_start["base"]
        
    df = build_dataframe(df_start['all_links'])
    df = df.dropna(subset=['description'])
    df = df[df['title'] != 'N/A'].copy()
    df = format_price_and_surface(df)
        
    # Define the pattern to match URLs to be removed
    pattern = r"https://www\.athome\.lu/srp/\?tr=rent&q=[^&]+&ptypes=flat&page=\d+"

    # Filter out matching URLs and drop duplicates
    df_filtered = df[~df['link_url'].str.match(pattern)].drop_duplicates(subset=['link_url'])

    # Instead of displaying the result, return df_filtered
    return df_filtered
if __name__ == "__main__":
    df_filtered = main()
    # You can now use df_filtered for further processing
    print(df_filtered.shape)
    print(df_filtered.columns)
    print(df_filtered.head(10).T)


https://www.athome.lu/srp/?tr=rent&q=a2d9b00c&ptypes=flat&page=2
https://www.athome.lu/location/appartement/luxembourg/id-8249591.html
https://www.athome.lu/location/appartement/luxembourg/id-8256763.html
https://www.athome.lu/srp/?tr=rent&q=a2d9b00c&ptypes=flat&page=4
https://www.athome.lu/location/appartement/luxembourg/id-8239058.html
https://www.athome.lu/srp/?tr=rent&q=a2d9b00c&ptypes=flat&page=4
https://www.athome.lu/srp/?tr=rent&q=a2d9b00c&ptypes=flat&page=1
https://www.athome.lu/location/appartement/luxembourg/id-8221912.html
https://www.athome.lu/location/appartement/luxembourg/id-8258064.html
https://www.athome.lu/location/appartement/luxembourg/id-8153070.html
https://www.athome.lu/location/appartement/luxembourg/id-8230205.html
https://www.athome.lu/location/appartement/luxembourg/id-8258064.html
https://www.athome.lu/location/appartement/luxembourg/id-8230224.html
https://www.athome.lu/srp/?tr=rent&q=a2d9b00c&ptypes=flat&page=9
https://www.athome.lu/srp/?tr=rent&q=a2d9b00c

In [8]:
df_filtered.head(10).T

,1,2,4,7,8,9,10,12,15,17
link_url,https://www.athome.lu/location/appartement/luxembourg/id-8249591.html,https://www.athome.lu/location/appartement/luxembourg/id-8256763.html,https://www.athome.lu/location/appartement/luxembourg/id-8239058.html,https://www.athome.lu/location/appartement/luxembourg/id-8221912.html,https://www.athome.lu/location/appartement/luxembourg/id-8258064.html,https://www.athome.lu/location/appartement/luxembourg/id-8153070.html,https://www.athome.lu/location/appartement/luxembourg/id-8230205.html,https://www.athome.lu/location/appartement/luxembourg/id-8230224.html,https://www.athome.lu/location/appartement/luxembourg/id-8238942.html,https://www.athome.lu/location/appartement/luxembourg/id-8231767.html
title,Appartement 1 chambre à Luxembourg-Limpertsberg,Studio 1 chambre à Luxembourg-Limpertsberg,Studio à Luxembourg-Limpertsberg,Chambre à Luxembourg-Limpertsberg,Appartement 1 chambre à Luxembourg-Limpertsberg,Appartement 1 chambre à Luxembourg-Limpertsberg,Chambre à Luxembourg-Limpertsberg,Appartement 1 chambre à Luxembourg-Limpertsberg,Appartement 1 chambre à Luxembourg-Limpertsberg,Studio 1 chambre à Luxembourg-Limpertsberg
description,"+++LIMPERTSBERG AVENUE PASTEUR LOCALISATION EXCEPTIONNELLE!! AVAILABLE 01.03.2025 FULLY FURNISHED SHORT TERM COURT SEJOUR +++RARE! BIJOU A LIMPERTSBERG BY EUREKA REAL ESTATE+++EXCLUSIVITE magnifique appartement au sein d'une nouvelle résidence avenue PASTEUR dans le prestigieux quartier de LIMPERTSBERG, proche du centre ville, des transports en commun et des restaurants, qui se compose comme suit: \n\n-hall d'entrée avec domotique permettant d'ouvrir/fermer tous les stores et toutes les lumières et de régler le chauffage au sol \n-living entièrement équipée (INTERNET TELEVISION CANAPE RANGEMENTS TABLE CHAISES ETC) avec parquet massif au sol et chauffage au sol \n-cuisine BULTAUP faite par un architecte d'intérieur avec plan de travail en corian entièrement meublée et équipée avec tous les ustensiles de cuisine (poêles, casseroles, couverts etc) \n-chambre avec grand lit double et de très nombreux rangements intégrés sur mesure \n-accès à une grande terrasse de 20m2 exposée sud/ouest sans aucun vis à vis avec vue sur jardin Une grande cave, une machine à laver et un sèche linge viennent compléter cet ensemble \n\nC'EST UNE SOLUTION IDEALE POUR DU COURT SEJOUR IL N'Y A PLUS QU'A POSER SES VALISES TOUT EST INCLUS (INTERNET TELEVISION EAU CHAUFFAGE USTENSILES DE CUISINE MACHINE A LAVER SECHE LINGE ETC) \n\nPOSSIBILITE DE LOUER L'APPARTEMENT POUR UNE DUREE DE 6 MOIS \n\nDISPONIBLE 01.03.2025 \n\nA VISITER ABSOLUMENT COUP DE COEUR ASSURE!!! \n\nPour plus de renseignements et pour convenir d'une visite, merci de contacter le +352 671 137 400","Idéalement situé à Limpertsberg, ce studio se ttrouve dans une petite résidence avec ascenseur et bénéficie d'une surface totale de ± 34 m² dont ± 29 m² habitables. Il se compose comme suit :\n\nAu 1er étage le hall d'entrée de ± 3 m² dessert une salle de bains de ± 3 m² (baignoire, lavabo, wc) ainsi qu'un salon/chambre de ± 21 m² avec en suite une kitchenette de ± 2 m² et un balcon de ± 2 m² sans vis-à-vis.\n\nAu sous-sol une cave de ± 6 m² complète l'offre.\n\nGénéralités :\n\n- Localisation idéale\n- Environnement très calme\n- Chaudière au gaz, chauffage radiateurs\n- Interphone\n- Balcon\n- Écoles, crèches, parcs, aires de jeux, restaurants, commerces à proximité\n- Proche du centre-ville\n- Desservi par les transports en commun\n- Disponibilité immédiate\n- Loyer mensuel : 1700 €\n- Charges locatives : 150 €\n- Dépôt de garantie : 2 mois de loyer\n- Frais d'agence : 1 mois de loyer + TVA\n\nPersonne responsable du bien :\n\nYuliana Dimitrova\nTel: +352661 50 27 19\nemail: yuliana@vanmaurits.lu","+++LIMPERTSBERG AVENUE VICTOR HUGO LOCALISATION EXCEPTIONNELLE!! SHORT TERM COURT SEJOUR FULLY FURNISHED AVAILABLE 15.01.2025+++EUREKA REAL ESTATE vous présente en exclusivité à la location un magnifique studio entièrement REFAIT A NEUF et 